In [1]:
pip install prophet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from prophet import Prophet

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression

Importing plotly failed. Interactive plots will not work.


In [3]:
SPARK_MASTER_IP = '172.18.0.2' 
spark = SparkSession.builder.appName("pyspark-taxi-forecasting_stage2") \
    .master(f"spark://{SPARK_MASTER_IP}:7077") \
    .config('spark.local.dir', 'spark_tmp/') \
    .config("spark.executor.cores", 1) \
    .config("spark.task.cpus", 1) \
    .getOrCreate()

spark = SparkSession.builder.appName("Introduction to Spark").getOrCreate()

In [4]:
spark

# Создадим признаки задерки и скользящего среднего

In [5]:
all_hours = spark.read.csv("all_hours_onlycode.csv", header = True, inferSchema = True)

In [6]:
test = spark.read.csv("y_true_2023-07-31_23-00_UTC0.csv", header = True, inferSchema = True)
test = test.withColumn('ds', test['hours']) \
    .withColumn('pickup_community_area', test['Pickup Community Area']) \
    .withColumn('y', test['trips_count'])

In [7]:
test.show()
test.count()

+---------------------+-------------------+-----------+-------------------+---------------------+---+
|Pickup Community Area|              hours|trips_count|                 ds|pickup_community_area|  y|
+---------------------+-------------------+-----------+-------------------+---------------------+---+
|                    0|2023-07-31 23:00:00|         22|2023-07-31 23:00:00|                    0| 22|
|                    1|2023-07-31 23:00:00|          2|2023-07-31 23:00:00|                    1|  2|
|                    2|2023-07-31 23:00:00|          1|2023-07-31 23:00:00|                    2|  1|
|                    3|2023-07-31 23:00:00|          5|2023-07-31 23:00:00|                    3|  5|
|                    4|2023-07-31 23:00:00|          0|2023-07-31 23:00:00|                    4|  0|
|                    5|2023-07-31 23:00:00|          4|2023-07-31 23:00:00|                    5|  4|
|                    6|2023-07-31 23:00:00|         49|2023-07-31 23:00:00|       

78

In [8]:
column_list = ["pickup_community_area"]
  

Windowspec = Window.partitionBy(["pickup_community_area"]).orderBy(all_hours.hour_cons.desc())
  
all_hours_lagged = all_hours.withColumn(
    'med_cost_lagged1', lag(all_hours['cost_median'], -1).over(Windowspec)).withColumn(
    'med_miles_lagged1', lag(all_hours['miles_median'], -1).over(Windowspec)).withColumn(
    'med_seconds_lagged1', lag(all_hours['seconds_median'], -1).over(Windowspec)).withColumn(
    'trips_count_lagged1', lag(all_hours['trips_count'], -1).over(Windowspec)).withColumn(
    'trips_count_lagged2', lag(all_hours['trips_count'], -2).over(Windowspec)).withColumn(
    'trips_count_lagged3', lag(all_hours['trips_count'], -3).over(Windowspec)).withColumn(
    'trips_count_lagged12', lag(all_hours['trips_count'], -12).over(Windowspec)).withColumn(
    'trips_count_lagged24', lag(all_hours['trips_count'], -24).over(Windowspec)).withColumn(
    'trips_count_lagged_week', lag(all_hours['trips_count'], -24*7).over(Windowspec))

all_hours_lagged = all_hours_lagged.withColumn(
    'rolling_average_on3hours', avg(all_hours_lagged['trips_count_lagged1'],).over(Windowspec.rowsBetween(-3, -0))) \
    .withColumn('rolling_average_on24hours', avg(all_hours_lagged['trips_count_lagged1'],).over(Windowspec.rowsBetween(-24, -0))) \
    .na.drop('any').cache()


# Проведем предсказания с помощью модели Prophet

## Разделим данные для Prophet

In [9]:
train_proph = all_hours_lagged.filter((all_hours_lagged.hour_cons < '2023-07-31 23:00:00')) \
    .selectExpr( 'pickup_community_area',
    'hour_cons as ds', 'trips_count as y')
# Partition the data dfsp_partitionned
train_proph.createOrReplaceTempView("pickup_community_area")
sql = "select * from pickup_community_area"
train_proph = (spark.sql(sql)\
   .repartition(spark.sparkContext.defaultParallelism, 
   ['pickup_community_area'])).cache()
train_proph.explain()

== Physical Plan ==
InMemoryTableScan [pickup_community_area#18, ds#396, y#397]
   +- InMemoryRelation [pickup_community_area#18, ds#396, y#397], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- Exchange hashpartitioning(pickup_community_area#18, 4), REPARTITION_BY_NUM, [plan_id=157]
            +- *(1) Project [pickup_community_area#18, hour_cons#19 AS ds#396, trips_count#21 AS y#397]
               +- *(1) Filter (isnotnull(hour_cons#19) AND (hour_cons#19 < 2023-07-31 23:00:00))
                  +- InMemoryTableScan [hour_cons#19, pickup_community_area#18, trips_count#21], [isnotnull(hour_cons#19), (hour_cons#19 < 2023-07-31 23:00:00)]
                        +- InMemoryRelation [_c0#17, pickup_community_area#18, hour_cons#19, taxi_countdist#20, trips_count#21, cost_median#22, miles_median#23, seconds_median#24, med_cost_lagged1#114, med_miles_lagged1#125, med_seconds_lagged1#136, trips_count_lagged1#148, trips_count_lagged2#161, trips_count_lagged3#175, trips_count_

In [10]:
valid_proph = all_hours_lagged.filter(all_hours_lagged.hour_cons == '2023-07-31 23:00:00') \
    .selectExpr( 'pickup_community_area',
    'hour_cons as ds', 'trips_count as y') \
    .withColumn("y",col("y").cast(DoubleType()))

## Проведем обучение и предсказание модели через функцию Pandas_udf

In [11]:
# Define a schema
schema = StructType([ \
                     StructField('pickup_community_area', IntegerType()), 
                     StructField('ds', TimestampType()),
                     StructField('y', FloatType()),
                     StructField('yhat', DoubleType()),
                     StructField('daily', DoubleType()),
                     StructField('weekly', DoubleType())
                    ])

In [12]:
# define the Pandas UDF
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def apply_model(store_pd):  # instantiate the model and set parameters
  model = Prophet(
      interval_width=0.95,
      growth='linear',
      n_changepoints = 150,
      daily_seasonality=True,
      weekly_seasonality=True,
      yearly_seasonality=True,
      seasonality_mode='additive'
  )  # fit the model to historical data
  model.fit(store_pd)  # Create a data frame that lists 90 dates starting from Jan 1 2018
  future = model.make_future_dataframe(
      periods=1, #valid_proph.filter(valid_proph.pickup_community_area==community_num).count(),
      freq='h',
      include_history=True
   )  # Out of sample prediction
  prediction = model.predict(future)  # Create a data frame that contains store, item, y, and yhat
  f_pd = prediction[['ds', 'yhat', 'daily', 'weekly']]
  st_pd = store_pd[['ds', 'pickup_community_area', 'y']]
  result_pd = f_pd.join(st_pd.set_index('ds'), on='ds', how='left')  # fill store and item
  result_pd['pickup_community_area'] = store_pd['pickup_community_area'].iloc[0]
  return result_pd[['pickup_community_area', 'ds', 'y', 'yhat',
                    'daily', 'weekly']]# Apply the function to all store-items
# Print the results - calculate the time to run
results = train_proph.groupby(['pickup_community_area']).apply(apply_model).cache()


/usr/local/spark/python/pyspark/sql/pandas/group_ops.py:103: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


# Проведем обучение и предсказания

## Выделим тренировочную и тестовую выборки

In [13]:
train_lr = all_hours_lagged.filter((all_hours_lagged.hour_cons < '2023-07-31 23:00:00')) \
    .selectExpr( 'pickup_community_area',
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
    'hour_cons as ds', 'trips_count as y') \
        .join(results['pickup_community_area', 'ds', 'daily', 'weekly'], on=['ds','pickup_community_area'] , how='inner').cache()

In [14]:
test_lr = all_hours_lagged.filter(all_hours_lagged.hour_cons == '2023-07-31 23:00:00') \
    .selectExpr('pickup_community_area',
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
    'hour_cons as ds').join(     #'trips_count as y') \
    results['pickup_community_area', 'ds', 'daily', 'weekly'], on=['ds','pickup_community_area'] , how='inner').cache()

In [15]:
featureCols = [
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
     'daily', 'weekly'
]

## Создадим pipeline

In [16]:
# положить фичи в вектор
assembler = VectorAssembler(inputCols=featureCols, outputCol="features") 

In [17]:
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd = True)

In [18]:
lr = (LinearRegression(featuresCol='features_scaled', labelCol="y", predictionCol='y_pred')) 
#                               maxIter=10, regParam=0.3, elasticNetParam=0.8, standardization=False))

In [19]:
pipeline = Pipeline(stages=[assembler, standardScaler, lr])

In [20]:
evaluator = RegressionEvaluator(predictionCol="y_pred", labelCol='y', metricName='mae')

# Реализуем регрессию для каждого региона через функцию

In [21]:
def modelsandpredictions(func_community):
        
    temp_train = train_lr.filter(train_lr.pickup_community_area == func_community)
    temp_test = test_lr.filter(test_lr.pickup_community_area == func_community)
    
    temp_lr = pipeline.fit(temp_train)
    temp_pred = temp_lr.transform(temp_test)
    temp_predandtrue = temp_pred.select(
                                        'pickup_community_area',
                                        'ds',
                                        "y_pred", 
                                        ).withColumn("y_pred",F.round(temp_pred["y_pred"],0)) 
    temp_predandtrue = temp_predandtrue.withColumn('y_pred', F.when((F.col("y_pred") <= 0), 0)\
        .otherwise(temp_predandtrue.y_pred)).cache()
    #temp_lr = lr.fit(temp_train.union(temp_test))
    #temp_lr.write().overwrite().save("/models/lr{0}".format(i))
    return temp_predandtrue

In [22]:
preds_schema = StructType([ \
    StructField("pickup_community_area",IntegerType (),True), \
    StructField("ds",TimestampType(),True), \
    StructField("y_pred",FloatType(),True) \
  ])

In [23]:
%%time
test_preds = spark.createDataFrame([],preds_schema)
for i in range(78):
    test_preds = test_preds.union(modelsandpredictions(i))

CPU times: user 5.89 s, sys: 2.53 s, total: 8.43 s
Wall time: 12min 24s


In [24]:
test_preds.show(5)
test_preds.count()

+---------------------+-------------------+------+
|pickup_community_area|                 ds|y_pred|
+---------------------+-------------------+------+
|                    0|2023-07-31 23:00:00|   0.0|
|                    1|2023-07-31 23:00:00|   2.0|
|                    2|2023-07-31 23:00:00|   0.0|
|                    3|2023-07-31 23:00:00|   3.0|
|                    4|2023-07-31 23:00:00|   2.0|
+---------------------+-------------------+------+
only showing top 5 rows



78

In [25]:
trueandpreds = test.join(test_preds, on=['ds','pickup_community_area'], how='inner') \
    .select(
        test.ds,
        test.pickup_community_area,
        test.y,
        test_preds.y_pred).cache()

## Посмотрим финальные предсказания и оценим их

In [22]:
%%time
trueandpreds.sort(trueandpreds.pickup_community_area.desc()).show(80)
print("LinearRegression MAE: {0}".format(evaluator.evaluate(trueandpreds)))
print('MAPE:',
    trueandpreds.select(avg((100*abs((trueandpreds.y - trueandpreds.y_pred) / trueandpreds.y)))).collect())

+-------------------+---------------------+---+------+
|                 ds|pickup_community_area|  y|y_pred|
+-------------------+---------------------+---+------+
|2023-07-31 23:00:00|                   77|  2|   4.0|
|2023-07-31 23:00:00|                   76|160| 205.0|
|2023-07-31 23:00:00|                   75|  0|   3.0|
|2023-07-31 23:00:00|                   74|  0|   0.0|
|2023-07-31 23:00:00|                   73|  0|   0.0|
|2023-07-31 23:00:00|                   72|  0|   1.0|
|2023-07-31 23:00:00|                   71|  0|   2.0|
|2023-07-31 23:00:00|                   70|  0|   2.0|
|2023-07-31 23:00:00|                   69|  2|   0.0|
|2023-07-31 23:00:00|                   68|  0|   3.0|
|2023-07-31 23:00:00|                   67|  0|   1.0|
|2023-07-31 23:00:00|                   66|  2|   1.0|
|2023-07-31 23:00:00|                   65|  0|   1.0|
|2023-07-31 23:00:00|                   64|  0|   0.0|
|2023-07-31 23:00:00|                   63|  0|   0.0|
|2023-07-3

    Окончательные ошибки отдельных Линейных регрессий для каждого района составили
    МАЕ: 2.1025641025641026
    МАРЕ: 65.79573756314875

    Возможно, тестовые данные были обработаны иначе. Необходимо определить и скоординировать в частности, работу с пропусками и заполнение района 0 в тестовых и тренировочных данных.